In [76]:
import pandas as pd
from pathlib import Path
import json
import pandas as pd
from rapidfuzz import fuzz, process, utils
import pymongo

In [96]:
data = pd.read_csv('/Users/andrea/Documents/AIDA/Progetti/unisurf/data/raw_data/ListaDOCENTI_AteneiStatali.csv', header=0, sep=';')

In [3]:
print(data.head(5))

      Fascia          Cognome e Nome Genere                 Ateneo Facoltà  \
0  Associato     ABASTANTE Francesca      F  Politecnico di TORINO     NaN   
1  Ordinario       ABATANGELO Chiara      F                 PADOVA     NaN   
2  Associato        ABATANGELO Laura      F  Politecnico di MILANO     NaN   
3  Associato       ABATANGELO Nicola      M                BOLOGNA     NaN   
4  Ordinario  ABATE Andrea Francesco      M                SALERNO     NaN   

     GSD 2024   SSD 2024 SSD 2015 SC 2015  \
0  08/CEAR-03  CEAR-03/C  ICAR/22   08/A3   
1  12/GIUR-01  GIUR-01/A   IUS/01   12/A1   
2  01/MATH-03  MATH-03/A   MAT/05   01/A3   
3  01/MATH-03  MATH-03/A   MAT/05   01/A3   
4  01/INFO-01  INFO-01/A   INF/01   01/B1   

                              Struttura di afferenza  \
0  INTERATENEO DI SCIENZE, PROGETTO E POLITICHE D...   
1       DIRITTO PRIVATO E CRITICA DEL DIRITTO (DPCD)   
2                                         Matematica   
3                                   

In [4]:
data

,Fascia,Cognome e Nome,Genere,Ateneo,Facoltà,GSD 2024,SSD 2024,SSD 2015,SC 2015,Struttura di afferenza,Servizio prestato in altro ateneo
0,Associato,ABASTANTE Francesca,F,Politecnico di TORINO,NaN,08/CEAR-03,CEAR-03/C,ICAR/22,08/A3,"INTERATENEO DI SCIENZE, PROGETTO E POLITICHE D...",NaN
1,Ordinario,ABATANGELO Chiara,F,PADOVA,NaN,12/GIUR-01,GIUR-01/A,IUS/01,12/A1,DIRITTO PRIVATO E CRITICA DEL DIRITTO (DPCD),NaN
2,Associato,ABATANGELO Laura,F,Politecnico di MILANO,NaN,01/MATH-03,MATH-03/A,MAT/05,01/A3,Matematica,NaN
3,Associato,ABATANGELO Nicola,M,BOLOGNA,NaN,01/MATH-03,MATH-03/A,MAT/05,01/A3,Matematica,NaN
4,Ordinario,ABATE Andrea Francesco,M,SALERNO,NaN,01/INFO-01,INFO-01/A,INF/01,01/B1,Informatica,NaN
...,...,...,...,...,...,...,...,...,...,...,...
60196,Associato confermato,ZURZOLO Chiara,F,Napoli Federico II,NaN,05/BIOS-10,BIOS-10/A,BIO/13,05/F1,Medicina Molecolare e Biotecnologie Mediche,NaN
60197,Associato,ZUSSO Morena,F,PADOVA,NaN,05/BIOS-11,BIOS-11/A,BIO/14,05/G1,SCIENZE DEL FARMACO - DSF,NaN
60198,Associato,ZUZOLO Daniela,F,SANNIO di BENEVENTO,NaN,05/BIOS-01,BIOS-01/C,BIO/03,05/A1,Scienze e Tecnologie,NaN
60199,Ricercatore a t.d. - t.pieno (L. 79/2022),ZWERGEL Clemens Carl Thomas,M,"ROMA ""La Sapienza""",NaN,03/CHEM-07,CHEM-07/A,CHIM/11,03/D1,CHIMICA E TECNOLOGIE DEL FARMACO,NaN


In [5]:
df = data.Ateneo.value_counts()
print(df)

Ateneo
ROMA "La Sapienza"              3613
BOLOGNA                         3410
Napoli Federico II              3153
PADOVA                          2849
MILANO                          2586
                                ... 
Stranieri di PERUGIA              69
ROMA "Foro Italico"               68
I.U.S.S. - PAVIA                  45
Scuola Superiore Meridionale      16
CASD - Scuola Superiore           11
Name: count, Length: 69, dtype: int64


In [6]:
print(data.Ateneo.value_counts())

Ateneo
ROMA "La Sapienza"              3613
BOLOGNA                         3410
Napoli Federico II              3153
PADOVA                          2849
MILANO                          2586
                                ... 
Stranieri di PERUGIA              69
ROMA "Foro Italico"               68
I.U.S.S. - PAVIA                  45
Scuola Superiore Meridionale      16
CASD - Scuola Superiore           11
Name: count, Length: 69, dtype: int64


In [7]:
pd.set_option('display.max_rows', None)  # mostra tutte le righe



## Fase di matching

In [38]:
nomi_atenei = data["Ateneo"].unique()
len(nomi_atenei)

69

Ok, anche se gli atenei dovrebbero essere 61

In [40]:
BASE_DIR = Path().resolve()

In [41]:
ROOT = Path.cwd().parent.parent  # sali di 2 livelli
jsonl_path = ROOT / "data" / "raw_data" / "institutions_it.jsonl"

In [42]:
with open(jsonl_path, "r") as f:
    docs = [json.loads(line) for line in f]

In [ ]:
pwd

In [50]:
def trova_miglior_match(nome, docs, soglia=20):
    """
    Funzione che trova i match in base ai nomi alternativi di OpenAlex (display_name_alternatives) 
    """
    miglior_punteggio = 0
    miglior_doc = None
    miglior_alias = None

    for doc in docs:
        if doc.get("type") == 'funder' or doc.get("type") == 'education':
            for alias in doc.get("display_name_alternatives", []):
                score = fuzz.token_sort_ratio(nome.lower().strip(), alias.lower().strip())
                if score > miglior_punteggio:
                    miglior_punteggio = score
                    miglior_doc = doc
                    miglior_alias = alias

    if miglior_punteggio >= soglia:
        return {
            "nome_input": nome,
            "match_alias": miglior_alias,
            "score": miglior_punteggio,
            "id_documento": miglior_doc.get("id", None),  # o altro identificativo
            "nome_ufficiale": miglior_doc.get("display_name", None)
        }
    else:
        return {
            "nome_input": nome,
            "match_alias": None,
            "score": miglior_punteggio,
            "id_documento": None,
            "nome_ufficiale": None
        }

# Applichiamo la funzione a ogni nome
match_results = [trova_miglior_match(nome, docs) for nome in nomi_atenei]

df_match = pd.DataFrame(match_results)

In [51]:
df_match

,nome_input,match_alias,score,id_documento,nome_ufficiale
0,Politecnico di TORINO,Politecnico di Torino,100.000000,https://openalex.org/I177477856,Polytechnic University of Turin
1,PADOVA,INFN Padova,70.588235,https://openalex.org/I4210105455,"Istituto Nazionale di Fisica Nucleare, Sezione..."
2,Politecnico di MILANO,Politecnico di Bari,75.000000,https://openalex.org/I68618741,Polytechnic University of Bari
3,BOLOGNA,Universität Bologna,53.846154,https://openalex.org/I9360294,University of Bologna
4,SALERNO,Statale,57.142857,https://openalex.org/I189158943,University of Milan
5,Napoli Federico II,Universität Neapel Federico II,70.833333,https://openalex.org/I71267560,University of Naples Federico II
6,BARI,Menarini,50.000000,https://openalex.org/I4210108499,Menarini Group (Italy)
7,BRESCIA,Universität Brescia,53.846154,https://openalex.org/I79940851,University of Brescia
8,CATANIA,ABA Catanzaro,60.000000,https://openalex.org/I4210133179,Accademia di Belle Arti di Catanzaro
9,PISA,Universität Pisa,40.000000,https://openalex.org/I108290504,University of Pisa


Non funziona molto bene

In [52]:
print(docs[1].get("display_name_alternatives", []))

['Université de milan', 'Università degli Studi di Milano', 'Statale', 'Universität Mailand', 'Universitat de Milà']


In [53]:
print(docs[1].get("display_name"))

University of Milan


In [ ]:
from rapidfuzz import fuzz
from typing import Optional

def trova_display_name_atomico(nome_input: str, docs: list, soglia: int = 85) -> Optional[str]:
    """
    Trova il miglior display_name fuzzy-matching confrontando solo il campo 'display_name' di ciascun documento.

    Args:
        nome_input: nome da cercare (es. "MILANO")
        docs: lista di documenti con campo 'display_name'
        soglia: soglia minima di similarità per accettare un match

    Returns:
        Il display_name più simile, oppure None se nessuno supera la soglia.
    """
    miglior_score = 0
    miglior_display_name = None

    for doc in docs:
        if doc.get('type') == 'funder' or doc.get('type') == 'education': #selezioan solo i 'funder' ed education (dove dovrebbero risultare gli atenei)
            nome_doc = doc.get("display_name")
            score = fuzz.token_sort_ratio(nome_input.lower().strip(), nome_doc.lower().strip())
            if score > miglior_score:
                miglior_score = score
                miglior_display_name = doc

    if miglior_score >= soglia:
        return {
            "nome_input": nome_input,
            "matched_display_name": miglior_display_name.get("display_name"),
            "score": miglior_score,
            "id_documento": miglior_display_name.get("id"),
        }
    else:
        return {
            "nome_input": nome_input,
            "matched_display_name": None,
            "score": miglior_score,
            "id_documento": None,
        }


In [35]:
match_results = [trova_display_name_atomico(nome, docs, 0) for nome in nomi_atenei]

df_matches = pd.DataFrame(match_results)

In [36]:
df_matches

,nome_input,matched_display_name,score,id_documento
0,Politecnico di TORINO,Politecnico di Milano,66.666667,https://openalex.org/I93860229
1,PADOVA,Pegaso University,34.782609,https://openalex.org/I4210125310
2,Politecnico di MILANO,Politecnico di Milano,100.000000,https://openalex.org/I93860229
3,BOLOGNA,University of Bologna,50.000000,https://openalex.org/I9360294
4,SALERNO,University of Salerno,50.000000,https://openalex.org/I131729948
5,Napoli Federico II,University of Naples Federico II,68.000000,https://openalex.org/I71267560
6,BARI,Ri.MED,40.000000,https://openalex.org/I4210150906
7,BRESCIA,University of Brescia,50.000000,https://openalex.org/I79940851
8,CATANIA,University of Catania,50.000000,https://openalex.org/I39063666
9,PISA,University of Pisa,36.363636,https://openalex.org/I108290504


AttributeError: 'str' object has no attribute 'get'

In [23]:
funders = []
other = []
for doc in docs:
    if doc.get("type") == 'funder':
        funders.append(doc.get("display_name"))
    else:
        other.append(doc.get("display_name"))
#print(funders)
#print(other)
print(len(funders))
print(len(other))

133
1880


In [76]:
len(funders)

133

In [55]:
print(docs[1])

{'id': 'https://openalex.org/I189158943', 'ror': 'https://ror.org/00wjc7c48', 'display_name': 'University of Milan', 'country_code': 'IT', 'type': 'funder', 'type_id': 'https://openalex.org/institution-types/funder', 'lineage': ['https://openalex.org/I189158943'], 'homepage_url': 'https://www.unimi.it', 'image_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Festa%20del%20Perdono%20Facciata%20Milano.jpg', 'image_thumbnail_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Festa%20del%20Perdono%20Facciata%20Milano.jpg&width=300', 'display_name_acronyms': ['UNIMI'], 'display_name_alternatives': ['Université de milan', 'Università degli Studi di Milano', 'Statale', 'Universität Mailand', 'Universitat de Milà'], 'repositories': [{'id': 'https://openalex.org/S4306401627', 'display_name': 'di/segni (Università degli Studi di Milano)', 'host_organization': 'https://openalex.org/I189158943', 'host_organization_name': 'University of Milan', 

In [65]:
choices = [x for x in docs[1].get("display_name_alternatives")]
choices.insert(0, docs[1].get("display_name")) 
#choices.append(docs[1].get("display_name_alternatives"))

print(choices)

['University of Milan', 'Université de milan', 'Università degli Studi di Milano', 'Statale', 'Universität Mailand', 'Universitat de Milà']


In [67]:
process.extractOne("Università Statale di Milano", choices, scorer=fuzz.ratio)


('Università degli Studi di Milano', 76.66666666666666, 2)

# Fase Matching - parte 2

Step 1: match tra i nomi troncati delle università nella rubrica dei professori e i nomi operativi nella lista di atenei, in modo da associare il codice ateneo.

In [72]:
atenei_file = pd.read_csv("/Users/andrea/Documents/AIDA/Progetti/unisurf/data/raw_data/ListaAteneiMUR.csv")

In [73]:
atenei_file

,_id,COD_Ateneo,NomeEsteso,NomeOperativo,status,Tipologia_ateneo_descrizione,Statale/NonStataleLibera,Indirizzo,COMUNE,PROVINCIA,REGIONE,Zona Geografica,URL,Classe dimensione
0,1,101,Università degli studi di Torino,Torino,Attivo,Università,Statale,"Via Verdi, 8",TORINO,TORINO,PIEMONTE,NORD-OVEST,www.unito.it,60.000 e oltre
1,2,102,Politecnico di Torino,Torino Politecnico,Attivo,Politecnico,Statale,"Corso Duca degli Abruzzi, 24",TORINO,TORINO,PIEMONTE,NORD-OVEST,www.polito.it,da 30.000 a 59.999
2,3,103,Istituto Superiore di Educazione Fisica di Torino,Torino ISEF,Non attivo,ISEF,Non Statale,NaN,TORINO,TORINO,PIEMONTE,NORD-OVEST,NaN,NaN
3,4,201,Università degli studi del Piemonte orientale ...,Piemonte Orientale,Attivo,Università,Statale,"Via Duomo, 6",VERCELLI,VERCELLI,PIEMONTE,NORD-OVEST,www.uniupo.it,da 10.000 a 19.999
4,5,401,Università di Scienze Gastronomiche,Bra - Scienze Gastronomiche,Attivo,Università,Non Statale,"P.zza Vittorio Emanuele, 9",BRA,CUNEO,PIEMONTE,NORD-OVEST,www.unisg.it,inferiore a 10.000
5,6,701,Università della Valle d'Aosta,Aosta,Attivo,Università,Non Statale,"Strada Cappuccini, 2A",AOSTA,AOSTA,VALLE D'AOSTA,NORD-OVEST,www.univda.it,inferiore a 10.000
6,7,1001,Università degli studi di Genova,Genova,Attivo,Università,Statale,"Via Balbi, 5",GENOVA,GENOVA,LIGURIA,NORD-OVEST,www.unige.it,da 30.000 a 59.999
7,8,1201,"Università ""Carlo Cattaneo"" (LIUC)",Castellanza LIUC,Attivo,Università,Non Statale,"Corso Matteotti, 22",CASTELLANZA,VARESE,LOMBARDIA,NORD-OVEST,www.liuc.it,inferiore a 10.000
8,9,1202,Università degli Studi dell' Insubria,Insubria,Attivo,Università,Statale,"Via Ravasi, 2",VARESE,VARESE,LOMBARDIA,NORD-OVEST,www.uninsubria.it,da 10.000 a 19.999
9,10,1301,"Università telematica ""e-Campus"" di Novedrate ...",Novedrate e-Campus,Attivo,Università telematica,Non Statale,"Via Isimbardi, 10",NOVEDRATE,COMO,LOMBARDIA,NORD-OVEST,www.uniecampus.it,da 20.000 a 29.999


In [ ]:
for el in data.Ateneo:
    result = process.extractOne(el, atenei_file.NomeOperativo, scorer=fuzz.WRatio, processor=utils.default_process)
    print(el, result)

    if data.Ateneo == el:
        data["Cod_AteneoMatched"] = atenei_file[atenei_file["NomeOperativo"] == result[0]]["COD_Ateneo"]


Politecnico di TORINO ('Torino Politecnico', 95.0, 1)
PADOVA ('Padova', 100.0, 29)
Politecnico di MILANO ('Milano Politecnico', 95.0, 11)
BOLOGNA ('Bologna', 100.0, 35)
SALERNO ('Salerno', 100.0, 84)
Napoli Federico II ('Napoli Federico II', 100.0, 77)
BARI ('Bari', 100.0, 93)
PADOVA ('Padova', 100.0, 29)
BOLOGNA ('Bologna', 100.0, 35)
BRESCIA ('Brescia', 100.0, 21)
CATANIA ('Catania', 100.0, 106)
BRESCIA ('Brescia', 100.0, 21)
PISA ('Pisa', 100.0, 48)
MESSINA ('Messina', 100.0, 104)
ROMA "Tor Vergata" ('Roma Tor Vergata', 96.96969696969697, 58)
SALERNO ('Salerno', 100.0, 84)
BRESCIA ('Brescia', 100.0, 21)
Politecnico di MILANO ('Milano Politecnico', 95.0, 11)
ROMA "La Sapienza" ('Roma La Sapienza', 96.96969696969697, 57)
MOLISE ('Molise', 100.0, 91)
Napoli Federico II ('Napoli Federico II', 100.0, 77)
"Parthenope" di NAPOLI ('Napoli Parthenope', 95.0, 78)
CAMPANIA - "L. VANVITELLI" ("L'Aquila", 85.5, 85)
CATANIA ('Catania', 100.0, 106)
MOLISE ('Molise', 100.0, 91)
MESSINA ('Messina', 

In [92]:
print(len(atenei_file["NomeOperativo"]))
print(len(data["Ateneo"].unique()))


109
69


In [97]:
# Elenco originale
atenei = data["Ateneo"].dropna().unique()

# Crea lista di match
matches = []

for ateneo in atenei:
    match = process.extractOne(ateneo, atenei_file["NomeOperativo"], scorer=fuzz.WRatio, processor=utils.default_process)

    if match:
        nome_match, score, _ = match
        # cerca il codice corrispondente
        riga = atenei_file[atenei_file["NomeOperativo"] == nome_match]
        cod = riga["COD_Ateneo"].values[0] if not riga.empty else None
        matches.append({
            "COD_Ateneo": cod,
            "NomeRubrica": ateneo,
            "NomeOperativo": nome_match,
            "Score": int(score)   
        })

# Crea tabella ponte
df_ponte = pd.DataFrame(matches)

# Salva o ispeziona
df_ponte.to_csv("PonteMatchAtenei.csv", index=False)

In [98]:
# LISTA DOCENTI ATENEI NON STATALI
data = pd.read_csv("/Users/andrea/Documents/AIDA/Progetti/unisurf/data/raw_data/ListaDOCENTI_AteneiNonStatali.csv", sep=';', header=0)

atenei = data["Ateneo"].dropna().unique()

# Crea lista di match
matches = []

for ateneo in atenei:
    match = process.extractOne(ateneo, atenei_file["NomeOperativo"], scorer=fuzz.WRatio, processor=utils.default_process)

    if match:
        nome_match, score, _ = match
        # cerca il codice corrispondente
        riga = atenei_file[atenei_file["NomeOperativo"] == nome_match]
        cod = riga["COD_Ateneo"].values[0] if not riga.empty else None
        matches.append({
            "COD_Ateneo": cod,
            "NomeRubrica": ateneo,
            "NomeOperativo": nome_match,
            "Score": int(score)   
        })

# Crea tabella ponte
df_ponte = pd.DataFrame(matches)

# Salva o ispeziona
df_ponte.to_csv("PonteMatchAteneiNonStatali.csv", index=False)